In [1]:
!pip install pyodbc

In [2]:
import pandas as pd 
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
server = "DESKTOP-PGQN0G5"
database = "Project_MarketingAnalytics"
driver = "{ODBC Driver 17 for SQL Server}"

try:
    conn = pyodbc.connect(
        f"DRIVER={driver};"
        f"SERVER={server};"
        f"DATABASE={database};"
        "Trusted_Connection=yes;"
    )
    print("Successfully connected to the database")
except Exception as e:
    print(f"Connection error: {e}")


Successfully connected to the database


In [8]:
sql = "SELECT * FROM [dbo].[customer_reviews]"

In [9]:
customer_reviews_df = pd.read_sql_query(sql, conn)

customer_reviews_df.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_6512\2626322257.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_reviews_df = pd.read_sql_query(sql, conn)


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [10]:
sia=SentimentIntensityAnalyzer()

In [11]:
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [12]:
def categorize_sentiment(score, rating):
    if score > 0.05:  
        if rating >= 4:
            return 'Positive' 
        elif rating == 3:
            return 'Mixed Positive' 
        else:
            return 'Mixed Negative'  
    elif score < -0.05:  
        if rating <= 2:
            return 'Negative' 
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive' 
    else: 
        if rating >= 4:
            return 'Positive' 
        elif rating <= 2:
            return 'Negative'  
        else:
            return 'Neutral' 

In [13]:

def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0' 
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  
    else:
        return '-1.0 to -0.5'  

customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1
)

In [14]:

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)
print(customer_reviews_df.head())
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  
